In [1]:
import os
import sys
import torch

# Add the parent folder of the current working directory to the Python path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from src.video.video_process import classify_video, get_consecutive_classes
from src.train.model import load_new_model

class_names = ['ApplyEyeMakeup',
'ApplyLipstick',
'BlowDryHair',
'BrushingTeeth',
'Haircut',
'Background']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_new_model(num_classes=5, pretrained=True, device='cuda', freeze_params=True)
checkpoint_path = "../checkpoints/UCF101-filtered-lr0.0001-nobackgroundclass/model_e_10.pth"
model.load_state_dict(torch.load(checkpoint_path))
model.eval()
model.to(device)

video_path ='../data/Raw/test-vid/test-actions-vid.mp4'
predicted_classes, probs = classify_video(video_path, model, device,single_class=False)

print("RAW OUTPUTS")
print("Predicted classes:")
print(predicted_classes)
print("Probabilities:")
print(probs)


consecutive_classes = get_consecutive_classes(predicted_classes, class_names)
print("Consecutive classifications:")
print(consecutive_classes)

RAW OUTPUTS
Predicted classes:
[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 2 0 0 0 0 0 0 2 1 2 2 2 2 1 1
 1 1 1 1 2 1 2 2 3 3 3 3 3 3 3 3 4 4 3 4 2 2 2 2 3 4 2 2 4 4 2 4 2 2 2 2 2
 2 2 2 4 2 2 4 4]
Probabilities:
[[4.53114092e-01 1.92299992e-01 9.51022729e-02 6.39703721e-02
  1.95513278e-01]
 [5.13569355e-01 1.71743229e-01 1.24217577e-01 3.72295938e-02
  1.53240263e-01]
 [3.97964031e-01 4.10957485e-01 8.34326446e-02 5.96066602e-02
  4.80391867e-02]
 [2.61801183e-01 5.82467079e-01 3.92463580e-02 6.93847314e-02
  4.71006073e-02]
 [2.57228911e-01 6.00718796e-01 4.00502011e-02 6.70938864e-02
  3.49082202e-02]
 [1.83907658e-01 7.02195406e-01 2.90005933e-02 5.93659915e-02
  2.55303197e-02]
 [1.21323712e-01 7.41714656e-01 3.10752224e-02 8.91969875e-02
  1.66894272e-02]
 [1.23813577e-01 7.43725061e-01 1.30043859e-02 1.03233919e-01
  1.62230413e-02]
 [1.86960801e-01 6.56755269e-01 2.00546887e-02 1.09927498e-01
  2.63016

In [3]:
from src.utils.utils import smooth_predictions
smoothed_classes, smoothed_probs = smooth_predictions(predicted_classes, probs, window_size=4)

In [4]:
consecutive_classes_smoothed = get_consecutive_classes(smoothed_classes, class_names)
smoothed_probs_consecutive_classes = get_consecutive_classes(smoothed_probs.argmax(axis=1), class_names)

print("Smoothed consecutive classifications:")
print(consecutive_classes_smoothed)

print("Smoothed probabilities classification:")
print(smoothed_probs_consecutive_classes)



Smoothed consecutive classifications:
[('ApplyEyeMakeup', 4), ('ApplyLipstick', 16), ('BlowDryHair', 2), ('BrushingTeeth', 35), ('BlowDryHair', 2), ('ApplyLipstick', 1), ('ApplyEyeMakeup', 7), ('ApplyLipstick', 3), ('BlowDryHair', 1), ('ApplyLipstick', 10), ('BlowDryHair', 3), ('BrushingTeeth', 10), ('BlowDryHair', 7), ('BrushingTeeth', 5), ('BlowDryHair', 10), ('BrushingTeeth', 3)]
Smoothed probabilities classification:
[('ApplyLipstick', 19), ('BrushingTeeth', 42), ('ApplyEyeMakeup', 5), ('BlowDryHair', 8), ('ApplyLipstick', 6), ('BrushingTeeth', 12), ('Haircut', 3), ('BlowDryHair', 24)]


In [5]:
from src.video.video_process import annotate_video_with_classes
# Input parameters
input_video = "../data/Raw/test-vid/test-actions-vid.mp4"  # Your source video
save_directory = "../outputs"
output_name = "nobackgroundclass-video-e10-lr10-4"

# Create output directory if needed
os.makedirs(save_directory, exist_ok=True)

# Process video
video_path, config_path = annotate_video_with_classes(
    input_video, smoothed_probs_consecutive_classes, save_directory, output_name
)

print(f"Annotated video saved to: {video_path}")
print(f"Configuration saved to: {config_path}")

Video FPS: 29.985853639496725, Width: 2048, Height: 1080, Total Frames: 1895
Annotated video saved to: ../outputs\nobackgroundclass-video-e10-lr10-4.mp4
Configuration saved to: ../outputs\nobackgroundclass-video-e10-lr10-4_actions_seconds.json
